# <font color="blue">Django course, part 2:</font>

In the first part, we built the data models and the admin interface and so on. In this part, we are going to build a RESTful API for our application.

#### Setting up the project:

In DataGrip, we will create a new dabase called "storeonline". To do so, open a new Query Console, and type:

> CREATE DATABASE storeonline

So, we need to open our project and run our migrations. Then, Django will automatically create our database tables for us. Use the attached Zip file where you can find all the source codes and database seed file. 

Press "ctrl + t" and jump to the DATABASE section of the settings module. Change 'NAME': 'storeonline'. Save the changes and open the terminal:

$ pipenv install

to install all the dependencies. 

$ pipenv shell

$ python manage.py migrate

Back to DataGrip, close all the sessions, open a new Query console for storeonline. Back to zip file, go to the data folder and open the seed file in the DataGrip. Then, from the session drop-down (top right), select console_23, and in schema, select storeonline. Then, select all the statements and run them. Final step is to run from the webserver:

$ python manage.py runserver

For our new inserted database, create a new supperuser

$ python manage.py createsuperuser

## <font color="blue">2-Building RESTfull APIs</font>

#### What are RESTful APIs?

We need a way to expose our data to clients like a web app running inside a browser or a mobile app running on a mobile device. And this is where APIs (Application Programming Interface) come to the picture. Building an API is essentially building an interface that client apps can use to get or save the data. APIs are going to have a bunch of endpoints for different purposes. So, we can have an end point for getting the list of products as well as creating, updating, and deleting them. We can have other endpoints for managing our shopping carts, orders, and so on. Client apps can send requests to these endpoints and get or save products, orders, shopping carts, etc.. 

What is RESTful API?

REST is short for "Representational State Transfer" and stands for defining a bunch of ruls for clients and servers to communicate over the web. following these ruls help us build sysems that are fast, scalable, relaiable, easy to understand, and easy to change. An API that confirm these ruls called RESTful. There are three concepts that everyone shoul know: resources, resource representations, and HTTP methods.

#### Resources:

Resources in a RESTful API is objects in our application like Product, Cart, Collection. These resources are avalilable on the web and client applicaton can access them using URLs (Uniform Resource Locator). So, URL is way to locate a resource on the web. It is basically a web address. Assuming our website is hosted, for example, at http://rezabuy.com, we can access all products in  http://rezabuy.com/products or access an individual product http://rezabuy.com/products/1. A resource can contain other resources like reviews http://rezabuy.com/products/1/reviews or for an especific review http://rezabuy.com/products/1/reviews/1. As a general rule of thumb, we should not nest resources too deep, say two rows is fine, but beyond that the URL can get complex and unmanagable. The above pattern or consistency across these URLs is one of the attributes of RESTful APIs. So, if we simply follow this pattern are API will be familiar and easy to understand to others. 

#### Resource representations:

We can identify a resource using it's URL. When we hit that URL, the server returns the resource in a certain format or representation. Different way of representations are HTML, XML (an old format for representing data), or JSON (a modern format that replaces XML). None of these are an internal representation of resources on the server. In other words, on the server, we identify a resource like Product, using an object or an instance of a python class. But, we return this to the client, we represent it as a HTML, XML, or JSON. Because these are the formats that clients can understand. REST does not dictate any format or representation we should use. JSON stands for JavaScripts Object Notation. Here you see a JSON representation of a product that uses a dictionary (key-value pairs) where keis are always string format (should be surrounded by double quote) but the value can be any format like string, number, boolan, an another object (in the curly braces {}), array (like []) and so on:

In [ ]:
{
    "name":"Reza",
    "age": 37,
    "is_online": True,
    "employere":{},
    "interests":[]
}

#### HTTP methods:

when building RESTful APIs, we expose one or more endpoints for clients. each endpoind may support various kinds of operations. So, some endpoints may only allow reading data, while others may allow modifying data as well. This is where HTTP method comes to the picture. Using HTTP methods, the client can tell the server what it wants to do with the resource. So, HTTP defines method like "GET" for getting a resource or a collection of resources, "POST" for creating a resource, "PUT" for updating the resource, "PATCH" for updating a part of the resource like a subset of a properties, and "DELETE" for deleting a resource. For example, we are going to create a product. In our client app, we send a "POST /product" request to the products endpoint. So, the server knows we want to create a product. But, where is the product we want to create? We insert this product like a JSON object in the body of the request:

> {
> "title": "...",
> "price": 10
>}

The server will read this product and create it. Now, let's we want to update our product. First, we need to ask we want to update all the properties or just a subset of that. If we want to update all properties, we sould send a "PUT" request. Otherwise, we should send a "PATCH" request to the related URL (like /products/1). Similar to creating a product, we should include a product object in the body of the request in a JSON format. So, server will extract this product from the request and update it accordingly. For deleting a product, we should send a "DELETE /product/1" request, but, here no need to send a JSON object in the body of the request.

#### Installing Django REST framework:

In terminal:

$ pipenv install djangorestframework

Now, we need to add it in the installed_apps in the settings module.

> 'rest_framework',

#### Creating API views: